# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from time import time

from data import FESDDataset
from data import Frame, AugmentationParams
import json
import numpy as np

import cv2

from model import FESD
import copy

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from utils import AvgMeter, clip_gradient, get_scheduler

from tqdm import tqdm

num_gpus = torch.cuda.device_count()
print(f"Num cuda GPUs: {num_gpus}")

h:\FESD\FESDModel\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num cuda GPUs: 1


In [3]:
RECORDING_DIR = Path('H:/Recordings/')
CHECKPOINT_DIR = Path('checkpoints')

## Data Loading

Firstly we need to import all the recordings into the notebook.


In [4]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

In [5]:
batchsize = 10
train_size = 352

dataset = FESDDataset(RECORDING_DIR, train_size)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize)
# build dataloader
n_data = len(train_loader.dataset)
CE = torch.nn.BCEWithLogitsLoss().cuda()

Recordings Found: 25
Total Frames: 7500


In [17]:
dataset.randomize_augmentation_params = False
dataset.reset_augmentation_params()
rgb, depth, pose_2d, errors = dataset[0]
dataset.frame.show()
print(rgb.shape)
print(depth.shape)
print("All Missing Joints")
print(errors[errors==1])

print("All Wrong Joints")
print(pose_2d[errors==2])

(480, 480, 3)
torch.Size([480, 480, 3])
torch.Size([480, 480, 3])
All Missing Joints
tensor([1, 1, 1, 1, 1], dtype=torch.int8)
All Wrong Joints
tensor([[ -1.4217, 167.1556,   0.2603],
        [ -1.1519, 170.9957,   0.2511]], dtype=torch.float64)


### Build Model

The model proposed by RD3D is based on resnet50 so we copy a pretrained resnet50 model

In [7]:
resnet = torchvision.models.resnet50(pretrained=True)

model = FESD(32, copy.deepcopy(resnet))
print(model)

model = nn.DataParallel(model).cuda()

h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
h:\FESD\FESDModel\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FESD(
  (rd3d): RD3D(
    (resnet): I3DResNet(
      (conv1): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=(1, 1, 1), ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck3d(
          (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

## Train Model

In the following we define the training function and train a network on the training data.

In [21]:
# training
def train(train_loader, model, optimizer, criterion, scheduler, epoch, epochs):
    # multi-scale training  

    #model.train()
    loss_record = AvgMeter()
    for i, pack in enumerate(tqdm(train_loader), start=1):
        optimizer.zero_grad()
        
        rgbs, depths, poses_2d, errors = pack
        
        # RD3D
        rgbs = rgbs.cuda()
        depths = depths.cuda()
        print(rgbs.shape)
        print(depths.shape)
        rgbs = rgbs.unsqueeze(2)
        depths = depths.unsqueeze(2)

        images = torch.cat([rgbs, depths], 2).permute(0, 4, 2, 3, 1)
        
        # Seperate Network
        poses = pose_2d.cuda()

        gt = errors.cuda()
        
        pred_s = model(images, poses)
        # TODO Calculate different loss based on the error label
        loss = criterion(pred_s, gt)
        
        loss.backward()
        clip_gradient(optimizer, clip)
        optimizer.step()
        scheduler.step()

        if rate == 1:
            loss_record.update(loss.data, opt.batchsize)

        if i % 100 == 0 or i == len(train_loader) or True:
            print('Epoch [{:03d}/{:03d}], Step [{:04d}/{:04d}], Loss: {:.4f}'.
                        format(epoch, epochs, i, len(train_loader),
                               loss_record.show()))
    #     images = images.cuda()
    #     gts = gts.cuda()
    #     depths = depths.cuda()

    #     # multi-scale training samples
    #     trainsize = int(round(opt.trainsize * rate / 32) * 32)
    #     if rate != 1:
    #         images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
    #         images = images.unsqueeze(2)
    #         gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)

    #         depths = F.upsample(depths, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
    #         depths = depths.unsqueeze(2)
    #         images = torch.cat([images, depths], 2)

    #     if rate == 1:
    #         images = images.unsqueeze(2)
    #         depths = depths.unsqueeze(2)
    #         images = torch.cat([images, depths], 2)

    #     # forward
    #     pred_s = model(images)
    #     # TODO Calculate different loss based on the error label
    #     loss = criterion(pred_s, gts)

    #     loss.backward()
    #     clip_gradient(optimizer, opt.clip)
    #     optimizer.step()
    #     scheduler.step()
    #     if rate == 1:
            
    #         loss_record.update(loss.data, opt.batchsize)
      

In [9]:
# epoch number
epochs = 50
# optimizer
optim = 'adamW'
# learning rate
learning_rate = 0.000125
# learning rate scheduler. can be step, poly or cosine
lr_scheduler = 'cosine'
# warmup epoch
warmup_epoch = -1
# warmup multiplier
warmup_multiplier = 100
# for step scheduler. where to decay lr, can be a list
lr_decay_epochs = [120, 160, 200]
# for step scheduler. step size to decay lr
lr_decay_steps = 20 
# for step scheduler. decay rate for learning rate
lr_decay_rate = 0.1
# weight decay
weight_decay = 0.0001
# momentum for SGD
momentum = 0.9
# gradient clipping margin
clip = 0.5

In [10]:
if optim == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=weight_decay)
elif optim == 'adamW':
    optimizer = torch.optim.AdamW(model.parameters(), learning_rate, weight_decay=weight_decay)
elif optim == 'sdg':
    optimizer = torch.optim.SGD(model.parameters(), learning_rate / 10.0 * batchsize, momentum=momentum, weight_decay=weight_decay)

scheduler = get_scheduler(optimizer, len(train_loader), lr_scheduler, lr_decay_epochs, lr_decay_steps, lr_decay_rate, epochs, warmup_epoch, warmup_multiplier)

In [22]:
# routine
for epoch in range(1, epochs + 1):
    tic = time()
    train(train_loader, model, optimizer, CE, scheduler, epoch, epochs)
    print(f'epoch {epoch}, total time {time() - tic:.2f}, learning_rate {optimizer.param_groups[0]["lr"]}')

    if (epoch) % 10 == 0:
        # torch.save(model.state_dict(), os.path.join(opt.output_dir, f"RD3D_{epoch}_ckpt.pth"))
        print(f'epoch {epoch}, total time {time() - tic:.2f}, learning_rate {optimizer.param_groups[0]["lr"]}')

        print("checkpoint saved {}!".format(os.path.join(opt.output_dir, f"{epoch}_ckpt.pth")))
# torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"last_ckpt.pth"))
print(f"model saved {os.path.join(CHECKPOINT_DIR, f'last_ckpt.pth')}!")
checkpoint = os.path.join(opt.output_dir, f"last_ckpt.pth")

  0%|          | 0/750 [00:00<?, ?it/s]

torch.Size([10, 480, 480, 3])
torch.Size([10, 480, 480, 3])


  0%|          | 0/750 [00:00<?, ?it/s]

torch.Size([10, 3, 2, 480, 480])
tensor([[[[[0.1294, 0.1333, 0.1098,  ..., 0.1059, 0.0902, 0.1020],
           [0.1647, 0.1530, 0.1255,  ..., 0.0823, 0.0863, 0.0745],
           [0.1960, 0.1530, 0.1333,  ..., 0.0784, 0.0784, 0.0549],
           ...,
           [0.3333, 0.3452, 0.3411,  ..., 0.1569, 0.1765, 0.1843],
           [0.3530, 0.3491, 0.3452,  ..., 0.1608, 0.1726, 0.1882],
           [0.3452, 0.3411, 0.3254,  ..., 0.1608, 0.1686, 0.1804]],

          [[2.1797, 2.1797, 2.1895,  ..., 3.0742, 3.0762, 3.0762],
           [2.1797, 2.1797, 2.1895,  ..., 3.0762, 3.0762, 3.0762],
           [0.0000, 2.1914, 2.1992,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [2.6074, 2.6074, 2.6074,  ..., 0.0000, 0.0000, 0.0000],
           [2.6074, 2.6074, 2.6094,  ..., 0.0000, 0.0000, 0.0000],
           [2.6074, 2.6074, 2.6094,  ..., 0.0000, 0.0000, 0.0000]]],


         [[[0.1451, 0.1765, 0.1843,  ..., 0.1216, 0.1216, 0.1530],
           [0.1412, 0.1804, 0.1921,  ..., 0.1294, 0.1372, 

RuntimeError: Input type (torch.cuda.HalfTensor) and weight type (torch.cuda.FloatTensor) should be the same